In [1]:
from pathlib import Path 
import pandas as pd

In [3]:
# stat_df = [pd.read_csv(Path(Path.home(), "Documents/xray/sample_bench/unit_tests/data/log_0.csv")), pd.read_csv(Path(Path.home(), "Documents/xray/sample_bench/unit_tests/data/log_1.csv"))]
# stat_df = pd.concat(stat_dfs)

stat_df = pd.read_csv(Path(Path.home(), "Documents/xray/sample_bench/data/test/log_2.csv"))

stat_df.head()

,Unnamed: 0,step,time,ff,xray,r_work,r_free,rmsd
0,0,0,0.000001,4506.716219,5.423054,0.302339,0.311671,0.000000
1,1,1,4.851931,578.320784,5.443791,0.308579,0.320890,0.031342
2,2,2,6.475618,359.158734,5.463851,0.314844,0.329872,0.043008
3,3,3,8.070607,254.319795,5.482057,0.320628,0.337490,0.053144
4,4,4,9.650779,194.066233,5.501312,0.327229,0.344253,0.062743


In [7]:
stat_df["r_free"].nsmallest(10)

0    0.311671
1    0.320890
2    0.329872
3    0.337490
4    0.344253
5    0.347803
6    0.355310
7    0.356963
8    0.362333
9    0.366118
Name: r_free, dtype: float64